In [1]:
import pandas as pd
import geopandas as gpd
import os
import fiona
import numpy as np
from datetime import datetime

In [2]:
if os.getenv('USERNAME')=='ywang':
    folder      = 'C:\\Users\\ywang\\Documents\\Files_for_Py\\UrbanSim_input_Zoning'
    input_dir   = os.path.join(folder, 'inputs')
    output_dir  = os.path.join(folder, 'outputs')

In [3]:
## Input
pz10 = pd.read_csv(input_dir + '\\2015_12_21_zoning_parcels.csv')
pz50 = pd.read_csv(input_dir + '\\\p10_pba50_attr_20200407.csv',usecols=['PARCEL_ID','geom_id_s','nodev','juris_id'])

In [4]:
print(pz10.shape)
print(len(pz10.zoning_id.unique()))
pz10.rename(columns={'nodev':'nodev_pba40'},inplace = True)
#display(pz10.loc[pz10.zoning_id.isnull()])

print(pz50.shape)
pz50['geom_id_s'] = pd.to_numeric(pz50['geom_id_s'])
pz50.rename(columns = {'geom_id_s':'geom_id'},inplace = True)

(1950733, 8)
3241
(1956208, 4)


In [5]:
merg = pz10[['geom_id','zoning_id','zoning','juris','prop','tablename','nodev_pba40']].merge(pz50, on = 'geom_id', how = 'left')
#display(merg.dtypes)

for i in ['zoning_id','PARCEL_ID','nodev']:
    try:
        merg[i] = merg[i].fillna(-1).astype(np.int64)
        merg[i].replace(-1, None, inplace = True)
    except:
        print(i)
        pass
display(merg.dtypes)

geom_id         int64
zoning_id       int64
zoning         object
juris           int64
prop            int64
tablename      object
nodev_pba40     int64
PARCEL_ID       int64
juris_id       object
nodev           int64
dtype: object

In [6]:
today = datetime.today().strftime('%Y_%m_%d')
merg.to_csv(output_dir + '\\'+today+'_zoning_parcels.csv')

In [7]:
nodev_pba40_1_pba50_0 = merg.loc[(merg['nodev_pba40'] == 1) & (merg['nodev'] == 0)]
#nodev_pba40_1_pba50_0
nodev_pba40_1_pba50_0.drop(columns = 'geom_id',inplace = True)
nodev_pba40_1_pba50_0.to_csv(output_dir + '\\'+today+'nodev_pba40_1_pba50_0.csv')

C:\Users\ywang\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [8]:
nodev_pba40_0_pba50_1 = merg.loc[(merg['nodev_pba40'] == 0) & (merg['nodev'] == 1)]
#nodev_pba40_0_pba50_1
nodev_pba40_0_pba50_1.drop(columns = 'geom_id',inplace = True)
nodev_pba40_0_pba50_1.to_csv(output_dir + '\\'+today+'nodev_pba40_0_pba50_1.csv')